In [12]:
!pip install transitions
from transitions import Machine

class VendingMachine(object):
    def __init__(self):
        self.balance = 0
        self.selected_item = None
        self.items = {
            'A1': {'name': 'Вода', 'price': 50},
            'A2': {'name': 'Шоколад', 'price': 70}
        }

    def _buy_item(self):
        if self.selected_item in self.items and self.balance >= self.items[self.selected_item]['price']:
            print(f"Покупаем товар {self.selected_item} за {self.items[self.selected_item]['price']}.")
            self.balance -= self.items[self.selected_item]['price']
            return True
        return False 
    
    def _show_error(self):
        if self.selected_item not in self.items:
            print("Нет такого товара!")
        elif self.balance < self.items[self.selected_item]['price']:
            print("Недостаточно денег!")
        else:
            print("Какая-то неизвестная ошибка :(")

    def _reset(self):
        """Сброс баланса и выбранного товара."""
        if self.balance > 0:
            print(f"Возвращено денег: {self.balance}")
        self.balance = 0
        self.selected_item = None

    def _insert_coin(self, amount):  # Переименованный метод!
        """Метод для внесения денег."""
        self.balance += amount
        print(f"Баланс пополнен на {amount}. Текущий баланс: {self.balance}")
    
    def _select_item(self, item):
        self.selected_item = item
        print(f"Выбран товар: {item}.")
        
    

# Инициализация автомата
machine = VendingMachine()

# Определение состояний и переходов
states = ['waiting', 'payment', 'item_selected', 'dispense', 'refund', 'error', 'cancelling']
transitions = [
    {
        'trigger': 'insert_coin',
        'source': ['waiting', 'payment'],
        'dest': 'payment',
        'after': '_insert_coin'  
    },
    {
        'trigger': 'select_item',
        'source': 'payment',
        'dest': 'item_selected',
        'after': '_select_item'},
    {
        'trigger': 'confirm_purchase',
        'source': 'item_selected',
        'dest': 'dispense',
        'conditions': '_buy_item',
        'after': 'give_change'
    },
    {
        'trigger': 'confirm_purchase',
        'source': 'item_selected',
        'dest': 'error',
        'unless': '_buy_item',
        'after': '_show_error'
    },
    {
        'trigger': 'cancel',
         'source': ['item_selected', 'error'],
         'dest': 'cancelling',
         'after': 'give_change'
    },
    {
        'trigger': 'give_change',
        'source': ['dispense', 'cancelling'],
        'dest': 'refund',
        'after': 'reset'
    },
    {'trigger': 'reset',
     'source': 'refund',
     'dest': 'waiting',
     'after': '_reset'}
]

# Создаем конечный автомат
fsm = Machine(model=machine, states=states, transitions=transitions, initial='waiting')

# Пример использования
print("--- Успешная покупка ---")
machine.insert_coin(50)  
machine.select_item('A1')     
machine.confirm_purchase()  
print(f"Текущее состояние: {machine.state}")

print("\n--- Сценарий ошибки ---")
machine.insert_coin(30)   
machine.select_item('A2')    
machine.confirm_purchase()  
print(f"Текущее состояние: {machine.state}")

print("\n--- Возврат денег ---")
machine.cancel()          
print(f"Текущее состояние: {machine.state}")

--- Успешная покупка ---
Баланс пополнен на 50. Текущий баланс: 50
Выбран товар: A1.
Покупаем товар A1 за 50.
Текущее состояние: waiting

--- Сценарий ошибки ---
Баланс пополнен на 30. Текущий баланс: 30
Выбран товар: A2.
Недостаточно денег!
Текущее состояние: error

--- Возврат денег ---
Возвращено денег: 30
Текущее состояние: waiting
